In [ ]:
!git clone https://github.com/johnPa02/ocr_receipts.git

In [ ]:
%cd /kaggle/working/ocr_receipts/key_info_extraction/PICK-pytorch

In [ ]:
!pip install torchtext==0.6

In [ ]:
!pip install allennlp

In [ ]:
!pip install overrides==3.0.0

In [ ]:
!pip install --upgrade huggingface-hub==0.24.0

In [ ]:
%%writefile /kaggle/working/ocr_receipts/key_info_extraction/PICK-pytorch/config.json
{
    "name": "PICK_Default",
    "run_id":"test",

    "distributed":true,
    "local_world_size":4,
    "local_rank":-1,

    "model_arch": {
        "type": "PICKModel",
        "args": {
            "embedding_kwargs":{
                "num_embeddings": -1,
                "embedding_dim": 512
            },
            "encoder_kwargs":{
                "char_embedding_dim":-1,
                "out_dim": 512,
                "nheaders": 4,
                "nlayers": 3,
                "feedforward_dim": 1024,
                "dropout": 0.1,
                "image_encoder": "resnet50",
                "roi_pooling_mode": "roi_align",
                "roi_pooling_size": [7,7]
            },
            "graph_kwargs":{
                "in_dim":-1,
                "out_dim":-1,
                "eta": 1,
                "gamma": 1,
                "learning_dim": 128,
                "num_layers": 2
            },
            "decoder_kwargs":{
                "bilstm_kwargs":{
                    "input_size": -1,
                     "hidden_size": 512,
                     "num_layers": 2,
                     "dropout": 0.1,
                     "bidirectional": true,
                     "batch_first": true

                },
                "mlp_kwargs":{
                     "in_dim": -1,
                     "out_dim": -1,
                    "dropout": 0.1
                },
                "crf_kwargs":{
                    "num_tags":-1
                }
            }
        }
    },

    "train_dataset": {
        "type": "PICKDataset",
        "args": {
            "files_name":"/kaggle/working/ocr_receipts/key_info_extraction/train_list.csv",
            "boxes_and_transcripts_folder":"boxes_and_transcripts",
            "images_folder":"/kaggle/working/ocr_receipts/data/rotated_images",
            "entities_folder":"entities",
            "iob_tagging_type":"box_level",
            "resized_image_size": [560, 784],
            "ignore_error": false
        }
    },
    "validation_dataset": {
        "type": "PICKDataset",
        "args": {
            "files_name":"/kaggle/working/ocr_receipts/key_info_extraction/val_list.csv",
            "boxes_and_transcripts_folder":"boxes_and_transcripts",
            "images_folder":"/kaggle/working/ocr_receipts/data/rotated_images",
            "entities_folder":"entities",
            "iob_tagging_type":"box_level",
            "resized_image_size": [560, 784],
            "ignore_error": false
        }
    },
    "train_data_loader": {
        "type": "DataLoader",
        "args":{
            "batch_size": 4,
            "shuffle": true,
            "drop_last": true,
            "num_workers": 8,
            "pin_memory":true
        }
    },
    "val_data_loader": {
          "type": "DataLoader",
          "args":{
              "batch_size": 4,
              "shuffle": false,
              "drop_last": false,
              "num_workers": 8,
              "pin_memory":true
          }
      },

    "optimizer": {
          "type": "Adam",
          "args":{
              "lr": 0.0001,
              "weight_decay": 0,
              "amsgrad": true
          }
    },
    "lr_scheduler": {
        "type": "StepLR",
        "args": {
            "step_size": 30,
            "gamma": 0.1
        }
    },

    "trainer": {
        "epochs": 100,
        "gl_loss_lambda": 0.01,
        "log_step_interval": 10,
        "val_step_interval": 50,

        "save_dir": "saved/",
        "save_period": 20,
        "log_verbosity": 2,

        "monitor": "max overall-mEF",
        "monitor_open": true,
        "early_stop": 40,

        "anomaly_detection": false,
        "tensorboard": false,

        "sync_batch_norm":true
    }
}


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python3 -m torch.distributed.launch --nnode=1 --node_rank=0 --nproc_per_node=1 \
--master_addr=127.0.0.1 --master_port=5555 \
train.py -c config.json --local_world_size 1